In [2]:
# Imports
import pandas as pd
from datetime import datetime
import numpy as np
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
import matplotlib.pyplot as plt
import statsmodels.stats.diagnostic as smd
from statsmodels.tsa.stattools import coint
import plotly.graph_objects as go
from statsmodels.iolib.summary2 import summary_col
from scipy.stats import t
import re
# from statsmodels.sandbox.regression.gmm import IV2SLS
from statsmodels.iolib.summary2 import summary_col
import statsmodels.graphics.tsaplots as sgt
from linearmodels.iv import IV2SLS
from scipy.stats import chi2

In [5]:
FOLDER = '/home/akimovh/rockets_feathers/eggs/' # enter the directory you are working in

### Downloading the data

In [7]:
egg_cpi = pd.read_excel(FOLDER + 'data/cpi_ppi/eggs/egg_cpi.xlsx')
egg_cpi = pd.melt(egg_cpi, 
                    id_vars=['Year'],
                    var_name='Month', 
                    value_name='cpi')
month_map = {
                'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4,
                'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8,
                'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
            }

egg_cpi['month_num'] = egg_cpi['Month'].map(month_map)

egg_cpi['date'] = pd.to_datetime(egg_cpi['Year'].astype(str) + '-' + egg_cpi['month_num'].astype(str) + '-01')
egg_cpi = egg_cpi[['date', 'cpi', 'month_num']].copy()
egg_cpi = egg_cpi.dropna()
egg_cpi = egg_cpi.sort_values(by = 'date', ignore_index = True)

In [8]:
egg_ppi = pd.read_excel(FOLDER + 'data/cpi_ppi/eggs/egg_ppi.xlsx')
egg_ppi = pd.melt(egg_ppi, 
                    id_vars=['Year'],
                    var_name='Month', 
                    value_name='ppi')
month_map = {
                'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4,
                'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8,
                'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
            }

egg_ppi['Month_Num'] = egg_ppi['Month'].map(month_map)

egg_ppi['date'] = pd.to_datetime(egg_ppi['Year'].astype(str) + '-' + egg_ppi['Month_Num'].astype(str) + '-01')
egg_ppi = egg_ppi[['date', 'ppi']].copy()
egg_ppi = egg_ppi.dropna()
egg_ppi = egg_ppi.sort_values(by = 'date', ignore_index = True)

In [9]:
death = pd.read_csv(FOLDER + 'data/cpi_ppi/eggs/chicken_death.csv')


month_map = {
                'JAN': 1, 'FEB': 2, 'MAR': 3, 'APR': 4,
                'MAY': 5, 'JUN': 6, 'JUL': 7, 'AUG': 8,
                'SEP': 9, 'OCT': 10, 'NOV': 11, 'DEC': 12
            }

death['Month_Num'] = death['Period'].map(month_map)

death['date'] = pd.to_datetime(death['Year'].astype(str) + '-' + death['Month_Num'].astype(str) + '-01')
death = death[['date', 'Value']].copy()
death = death.sort_values(by = 'date', ignore_index= True)
death = death.rename(columns = {'Value':'death'})

### Merging data together + create nessesary columns (lags)

In [10]:
# Merge data + create ln + normalize to jan 2008 be the reference date

df =  pd.merge(pd.merge(egg_cpi, egg_ppi, on = 'date'), death, on = 'date')

df['ln_cpi'] = np.log(df.cpi)
df['ln_ppi'] = np.log(df.ppi)

df['ln_cpi'] = df['ln_cpi'] - df.query("date == '2008-01-01'").ln_cpi[0]
df['ln_ppi'] = df['ln_ppi'] - df.query("date == '2008-01-01'").ln_ppi[0]


In [11]:
# Create needed vars

df['intercept'] = 1

df['cpi_lag1'] = df['ln_cpi'].shift(1)
df['ppi_lag1'] = df['ln_ppi'].shift(1)

df['d_cpi'] = df['ln_cpi'].diff()
df['d_ppi'] = df['ln_ppi'].diff()
# df['d_feed'] = df['ln_feed'].diff()

df['d_ppi_pos_lag0'] = df['d_ppi'].apply(lambda x: x if x > 0 else 0)
df['d_ppi_neg_lag0'] = df['d_ppi'].apply(lambda x: x if x < 0 else 0)
df['d_cpi_pos_lag0'] = df['d_cpi'].apply(lambda x: x if x > 0 else 0)
df['d_cpi_neg_lag0'] = df['d_cpi'].apply(lambda x: x if x < 0 else 0)
# df['d_feed_neg_lag0'] = df['d_feed'].apply(lambda x: x if x < 0 else 0)
# df['d_feed_pos_lag0'] = df['d_feed'].apply(lambda x: x if x > 0 else 0)
lag_cols = {}

for i in range(1, 13):
    lag_cols[f"d_ppi_pos_lag{i}"] = df["d_ppi_pos_lag0"].shift(i)
    lag_cols[f"d_ppi_neg_lag{i}"] = df["d_ppi_neg_lag0"].shift(i)
    lag_cols[f"d_cpi_pos_lag{i}"] = df["d_cpi_pos_lag0"].shift(i)
    lag_cols[f"d_cpi_neg_lag{i}"] = df["d_cpi_neg_lag0"].shift(i)
    lag_cols[f"death_lag_{i}"] = df.death.shift(i)
    # lag_cols[f"d_feed_neg_lag{i}"] = df['d_feed_neg_lag0'].shift(i)
    # lag_cols[f"d_feed_pos_lag{i}"] = df['d_feed_pos_lag0'].shift(i)

df = pd.concat([df, pd.DataFrame(lag_cols)], axis=1)
df = df.query("date > '2009-12-01'").copy()

# df['sin_time'] = np.sin(2 * np.pi * df['date'].dt.month / 12)
# df['cos_time'] = np.cos(2 * np.pi * df['date'].dt.month / 12)
month_dummies = pd.get_dummies(df['month_num'], prefix='m', drop_first=True).astype(int)
df = pd.concat([df, month_dummies], axis=1)
month_col = list(month_dummies.columns)

# First Plots

In [13]:
x = df.date

# UC Berkeley colors
berkeley_blue = "#002676"
california_gold = "#FDB515"

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=x,
    y=df.ln_cpi,
    mode="lines",
    name="CPI",
    line=dict(color=berkeley_blue, width=3.5)
))

fig.add_trace(go.Scatter(
    x=x,
    y=df.ln_ppi,
    mode="lines",
    name="PPI",
    line=dict(color=california_gold, width=3.5)
))

fig.update_layout(
    title="Logged PPI and CPI dynamics",
    xaxis_title="Date",
    yaxis_title="Value",

    # ECONOMETRICA / QJE STYLE
    font=dict(
        family="Times New Roman",
        size=14,
        color="black"
    ),
    template="simple_white",

    # Axes styling
    xaxis=dict(
        showgrid=True,
        gridcolor="lightgray",
        zeroline=False,
        linecolor="black",
        linewidth=1,
        mirror=True,
        ticks="outside",
        tickwidth=1,
        tickcolor="black"
    ),
    yaxis=dict(
        showgrid=True,
        gridcolor="lightgray",
        zeroline=False,
        linecolor="black",
        linewidth=1,
        mirror=True,
        ticks="outside",
        tickwidth=1,
        tickcolor="black"
    ),

    # Legend OUTSIDE (right)
    legend=dict(
        x=1.02,
        y=1,
        xanchor="left",
        yanchor="top",
        font=dict(size=15),
        bgcolor="rgba(255,255,255,0)",
        bordercolor="rgba(0,0,0,0)"
    ),

    margin=dict(r=150),
    width=850,
    height=520
)

fig.show()

fig.write_image(FOLDER + "plots/cpi_ppi/Eggs_CPI_PPI.png") #this line saves the plot


CPI is way smoother

### Pre-analysis robustness checks

Since we working with time series OLS might be ill-suited if our variables are cointegrated

First, test for stationarity

In [14]:
def check_stationarity(series, name):
    result = adfuller(series.dropna())
    print(f"--- {name} ---")
    print(f"ADF Statistic: {result[0]:.4f}")
    print(f"P-Value:       {result[1]:.4f}")
    if result[1] < 0.05:
        print(">> STATIONARY (I(0))")
    else:
        print(">> NON-STATIONARY (Likely I(1))")
    print("\n")

check_stationarity(df['ln_cpi'], "Log CPI (Levels)")
check_stationarity(df['ln_ppi'], "Log PPI (Levels)")

check_stationarity(df['ln_cpi'].diff(), "d_Log_CPI (Inflation)")
check_stationarity(df['ln_ppi'].diff(), "d_Log_PPI (Inflation)")

--- Log CPI (Levels) ---
ADF Statistic: -1.7219
P-Value:       0.4199
>> NON-STATIONARY (Likely I(1))


--- Log PPI (Levels) ---
ADF Statistic: -1.7386
P-Value:       0.4113
>> NON-STATIONARY (Likely I(1))


--- d_Log_CPI (Inflation) ---
ADF Statistic: -7.3889
P-Value:       0.0000
>> STATIONARY (I(0))


--- d_Log_PPI (Inflation) ---
ADF Statistic: -12.8423
P-Value:       0.0000
>> STATIONARY (I(0))




Good news - our variables both I(1), so we can perform Engle-Granger Two-Step Method to test for cointegration

In [15]:
from statsmodels.tsa.stattools import coint

score, pvalue, _ = coint(df['ln_ppi'], df['ln_cpi'], trend='c') 

print(f"Correct P-Value: {pvalue}")

Correct P-Value: 0.007793641641588456


P_values is lesss than 0.05, so the variables are cointegrated nd we should use Error Correction Model (ECM)

### Error Correction Model with IV

Selecting what number of lags to use

In [16]:
best_lag = 1
best_model = None
print(f"--- Testing Lag 1 (Base) ---")

endog_vars = ['d_ppi_pos_lag0', 'd_ppi_neg_lag0']
instruments = ['death', 'death_lag_1',  'death_lag_5', 'death_lag_11']
exog_controls = ['intercept', 'cpi_lag1', 'ppi_lag1'] + month_col
new_lag_cols = [f"d_ppi_pos_lag{1}", f"d_ppi_neg_lag{1}", f"d_cpi_pos_lag{1}", f"d_cpi_neg_lag{1}"]
exog_controls += new_lag_cols

dependent_var = df['d_cpi']
endog = df[endog_vars]
exog = df[exog_controls]
instruments_only = df[instruments]

mod_1 = IV2SLS(dependent_var, exog, endog, instruments_only).fit()
best_model = mod_1

# Create restriction matrix for the new lag columns
restriction_dict = {col: 0 for col in new_lag_cols}
f_test = mod_1.wald_test(formula=restriction_dict)
p_val = f_test.pval

print(f"Lag 1 F-test p-value: {p_val:.4f}")

max_lags = 12
for k in range(2, max_lags + 1):
        print(f"--- Testing Lag {k} ---")
        new_lag_cols = [f"d_ppi_pos_lag{k}", f"d_ppi_neg_lag{k}", f"d_cpi_pos_lag{k}", f"d_cpi_neg_lag{k}"]
        exog_controls += new_lag_cols

        current_exog = df[exog_controls]
        current_endog = df[endog_vars]
        current_instruments_only = df[instruments]
        
        mod_k = IV2SLS(dependent_var, current_exog, current_endog, current_instruments_only).fit()

        # Create restriction dictionary for the new lag columns
        restriction_dict = {col: 0 for col in new_lag_cols}
        f_test = mod_k.wald_test(formula=restriction_dict)
        p_val = f_test.pval

        print(f"Lag {k} F-test p-value: {p_val:.4f}")

        if p_val < 0.10:
            print(f"   -> Significant. Keeping Lag {k} and continuing.")
            best_lag = k
            best_model = mod_k
        else:
            print(f"   -> Not significant. Stopping selection.")
            break

exog_controls = [col for col in exog_controls if col not in new_lag_cols]

print(f"\n--- Final Selection ---")
print(f"Optimal Lag Length: {best_lag}")


--- Testing Lag 1 (Base) ---
Lag 1 F-test p-value: 0.0000
--- Testing Lag 2 ---
Lag 2 F-test p-value: 0.0710
   -> Significant. Keeping Lag 2 and continuing.
--- Testing Lag 3 ---
Lag 3 F-test p-value: 0.1475
   -> Not significant. Stopping selection.

--- Final Selection ---
Optimal Lag Length: 2


#### Some robustness checks

First stage inspection

In [17]:
best_model.first_stage

,d_ppi_pos_lag0,d_ppi_neg_lag0
R-squared,0.4191,0.4579
Partial R-squared,0.2764,0.1462
Shea's R-squared,0.1705,0.0902
Partial F-statistic,50.715,37.673
P-value (Partial F-stat),2.56e-10,1.309e-07
Partial F-stat Distn,chi2(4),chi2(4)
==========================,============,============
intercept,-0.1443,-0.2069
,(-3.3419),(-3.7886)
cpi_lag1,0.1484,-0.0846


First stage is good enough: F-stats are 50.7 and 37.7

Wu-Hausman test to check that our variables indeed endogenous

In [18]:
print(best_model.wu_hausman())

Wu-Hausman test of exogeneity
H0: All endogenous variables are exogenous
Statistic: 6.1573
P-value: 0.0027
Distributed: F(2,159)


They are endogenous, so we right to use IV

Check for autocorrelation

In [19]:
# Setup
df['resids'] = best_model.resids
n = len(df['resids'])
max_lags = 3

print(f"{'Lags':<6} {'LM Stat':<10} {'P-Value':<10} {'Conclusion'}")
print("-" * 50)

for p in range(1, max_lags + 1):
    # 1. Generate the specific list of lag columns for this iteration (1 to p)
    current_lags = []
    for i in range(1, p + 1):
        col_name = f"resids_lag{i}"
        df[col_name] = df['resids'].shift(i)
        current_lags.append(col_name)

    # 2. Slice the data (drop first p rows to handle NaNs)
    y_tmp = df['resids'][p:]
    # Z = Controls + Instruments + Lags
    X_tmp = df[exog_controls + instruments + current_lags][p:]
    
    # 3. Auxiliary Regression
    model_tmp = sm.OLS(y_tmp, X_tmp).fit()

    # 4. Calculate Stats
    # Stat = (N - p) * R^2
    stat = (n - p) * model_tmp.rsquared
    
    # Degrees of freedom = p (number of lags being tested)
    p_value = chi2.sf(stat, p)
    
    # 5. Interpretation
    if p_value < 0.05:
        conclusion = "Autocorrelation DETECTED"
    else:
        conclusion = "No Autocorrelation"

    print(f"{p:<6} {stat:<10.4f} {p_value:<10.4f} {conclusion}")

Lags   LM Stat    P-Value    Conclusion
--------------------------------------------------
1      3.5956     0.0579     No Autocorrelation
2      4.7490     0.0931     No Autocorrelation
3      4.7745     0.1891     No Autocorrelation


No autocorrelation

### Results

We passed all robustness checks -> can proceed to results

First, let's extract important parametrs for regression table

In [20]:
res = best_model 

# Extract parameters
theta = res.params['cpi_lag1']
theta_se = res.std_errors['cpi_lag1']

phi = - res.params['ppi_lag1'] / theta

# Delta-method variance of ratio: φ = -(β / θ)
beta = res.params['ppi_lag1']
beta_se = res.std_errors['ppi_lag1']

# Covariance terms
cov = res.cov.loc['ppi_lag1', 'cpi_lag1']

# delta method for φ = -beta/theta
phi_var = (1/theta**2)*beta_se**2 + (beta**2/theta**4)*theta_se**2 - (2*beta/(theta**3))*cov
phi_se = np.sqrt(phi_var)

# t-stat & p-value
phi_t = phi / phi_se
degree = res.df_resid
phi_p = 2 * (1 - t.cdf(abs(phi_t), degree))

# significance stars
def stars(p):
    if p < 0.001: return "***"
    elif p < 0.01: return "**"
    elif p < 0.05: return "*"
    elif p < 0.1: return "+"
    return ""

phi_star = stars(phi_p)

# confidence interval
phi_ci_low  = phi - 1.96 * phi_se
phi_ci_high = phi + 1.96 * phi_se

Let's print our result as Latex formula

In [21]:
# 1. Extract base results into a DataFrame
# ----------------------------------------
df_res = pd.DataFrame({
    'Parameter': res.params,
    'Std. Err.': res.std_errors,
    'T-stat': res.tstats,
    'P-value': res.pvalues,
    'Lower CI': res.conf_int().iloc[:, 0],
    'Upper CI': res.conf_int().iloc[:, 1]
})

# 2. Define the new computed row for Phi
phi_row = pd.DataFrame({
    'Parameter': [phi],
    'Std. Err.': [phi_se],
    'T-stat': [phi_t],
    'P-value': [phi_p],
    'Lower CI': [phi_ci_low],
    'Upper CI': [phi_ci_high]
}, index=['phi_row']) # Temporary index name

# Append phi to the main dataframe
df_res = pd.concat([df_res, phi_row])


# 4. Define the Exact Row Order
# ----------------------------------------
# This list defines the order of rows. 
# Any variable in the model NOT in this list will be dropped.
desired_order = [
    'intercept',
    'd_ppi_pos_lag0',  # Beta+ 0
    'd_ppi_neg_lag0',  # Beta- 0
    'd_ppi_pos_lag1',  # Beta+ 1
    'd_ppi_neg_lag1',  # Beta- 1
    'd_ppi_pos_lag2',  # Beta+ 2
    'd_ppi_neg_lag2',  # Beta- 2
    'd_cpi_pos_lag1',  # Gamma+ 1
    'd_cpi_neg_lag1',  # Gamma- 1
    'cpi_lag1',        # This will become Theta
    'phi_row',         # This will become Phi
    # 'sin_time',        # S_t
    # 'cos_time'         # C_t
]

# Filter and Sort
df_final = df_res.reindex(desired_order)

# 5. Define LaTeX Label Mappings
# ----------------------------------------
latex_labels = {
    'intercept': r'\textbf{Intercept}',
    'd_ppi_pos_lag0': r'\textbf{\boldmath $\beta^+_{0}$}',
    'd_ppi_neg_lag0': r'\textbf{\boldmath $\beta^-_{0}$}',
    'd_ppi_pos_lag1': r'\textbf{\boldmath $\beta^+_{1}$}',
    'd_ppi_neg_lag1': r'\textbf{\boldmath $\beta^-_{1}$}',
    'd_ppi_pos_lag2': r'\textbf{\boldmath $\beta^+_{2}$}',
    'd_ppi_neg_lag2': r'\textbf{\boldmath $\beta^-_{2}$}',
    'd_cpi_pos_lag1': r'\textbf{\boldmath $\gamma^+_{1}$}',
    'd_cpi_neg_lag1': r'\textbf{\boldmath $\gamma^-_{1}$}',
    'cpi_lag1':       r'\textbf{\boldmath $\theta$}',
    'phi_row':        r'\textbf{\boldmath $\phi_1$}',
    # 'sin_time':       r'\textbf{\boldmath $S_t$}',
    # 'cos_time':       r'\textbf{\boldmath $C_t$}'
}

# Apply index renaming
df_final.index = df_final.index.map(latex_labels)


def fmt_stars(val):
    if val < 0.001: return "***"
    elif val < 0.01: return "**"
    elif val < 0.05: return "*"
    elif val < 0.1: return "+"
    return ""

# Format columns for display
formatted_df = pd.DataFrame()
formatted_df['Parameter'] = df_final.apply(lambda x: f"{x['Parameter']:.4f}{fmt_stars(x['P-value'])}", axis=1)
formatted_df['Std. Err.'] = df_final['Std. Err.'].apply(lambda x: f"{x:.4f}")
formatted_df['T-stat'] = df_final['T-stat'].apply(lambda x: f"{x:.4f}")
formatted_df['P-value'] = df_final['P-value'].apply(lambda x: f"{x:.4f}")
formatted_df['Lower CI'] = df_final['Lower CI'].apply(lambda x: f"{x:.4f}")
formatted_df['Upper CI'] = df_final['Upper CI'].apply(lambda x: f"{x:.4f}")

# 7. Generate LaTeX
# ----------------------------------------
latex_output = formatted_df.to_latex(
    index=True,
    escape=False, # Important to render the math symbols in index
    column_format='l|c|c|c|c|c|c|'
)

# 8. Post-processing to match your exact styling
# ----------------------------------------
# Add the [H] and center tags as you had before
latex_output = "\\begin{center}\n" + latex_output.replace(
    r"\begin{tabular}{l|c|c|c|c|c|c|}", 
    r"\begin{tabular}[H]{l|c|c|c|c|c|c|}"
) + "\n\\end{center}"

# Print
print(latex_output)

\begin{center}
\begin{tabular}[H]{l|c|c|c|c|c|c|}
\toprule
 & Parameter & Std. Err. & T-stat & P-value & Lower CI & Upper CI \\
\midrule
\textbf{Intercept} & 0.0371** & 0.0122 & 3.0349 & 0.0024 & 0.0131 & 0.0611 \\
\textbf{\boldmath $\beta^+_{0}$} & 0.1989*** & 0.0361 & 5.5050 & 0.0000 & 0.1281 & 0.2697 \\
\textbf{\boldmath $\beta^-_{0}$} & 0.1114* & 0.0496 & 2.2451 & 0.0248 & 0.0141 & 0.2086 \\
\textbf{\boldmath $\beta^+_{1}$} & 0.1188*** & 0.0224 & 5.3083 & 0.0000 & 0.0749 & 0.1627 \\
\textbf{\boldmath $\beta^-_{1}$} & 0.1042*** & 0.0207 & 5.0249 & 0.0000 & 0.0636 & 0.1449 \\
\textbf{\boldmath $\beta^+_{2}$} & 0.0397+ & 0.0221 & 1.7932 & 0.0729 & -0.0037 & 0.0830 \\
\textbf{\boldmath $\beta^-_{2}$} & 0.0322+ & 0.0195 & 1.6530 & 0.0983 & -0.0060 & 0.0705 \\
\textbf{\boldmath $\gamma^+_{1}$} & -0.2611** & 0.0942 & -2.7703 & 0.0056 & -0.4458 & -0.0764 \\
\textbf{\boldmath $\gamma^-_{1}$} & -0.2385* & 0.1072 & -2.2251 & 0.0261 & -0.4486 & -0.0284 \\
\textbf{\boldmath $\theta$} & -0.1489*

Let's compute commulative adjustment function

In [22]:
params = best_model.params
    
beta_pos = {}  # PPI coefficients
gamma_pos = {} # CPI positive feedback
gamma_neg = {} # CPI negative feedback
beta_neg = {}

periods = 12 # up to a year
    
for i in range(0, periods):
    beta_pos[i] = params.get(f'd_ppi_pos_lag{i}', 0)
    gamma_pos[i] = params.get(f'd_cpi_pos_lag{i}', 0)
    gamma_neg[i] = params.get(f'd_cpi_neg_lag{i}', 0)
    beta_neg[i] = params.get(f'd_ppi_neg_lag{i}', 0)
     

In [23]:
B_plus = [beta_pos[0]]
B_minus = [beta_neg[0]]


for k in range(1, periods):
    term_beta_pos = beta_pos[k]
    term_ect_pos = theta * (B_plus[k-1] - phi)
    term_ar = 0
    for j in range(1, best_lag + 1):
        if k - j >= 0:
            if k - j == 0:
                prev_change = B_plus[0]
            else:
                prev_change = B_plus[k-j] - B_plus[k-j-1]
            if prev_change > 0:
                term_ar += gamma_pos[j] * prev_change
            else:
                term_ar += gamma_neg[j] * prev_change
    delta_R_k = term_beta_pos + term_ect_pos + term_ar
    B_plus.append(B_plus[k-1] + delta_R_k)
    
    term_beta_neg = beta_neg[k]
    term_ect_neg = theta * (B_minus[k-1] - phi)
    term_ar = 0
    for j in range(1, best_lag + 1):
        if k - j >= 0:
            if k - j == 0:
                prev_change = B_minus[0]
            else:
                prev_change = B_minus[k-j] - B_minus[k-j-1]
            if prev_change > 0:
                term_ar += gamma_pos[j] * prev_change
            else:
                term_ar += gamma_neg[j] * prev_change
    delta_R_k = term_beta_neg + term_ect_neg + term_ar
    B_minus.append(B_minus[k-1] + delta_R_k)

B_plus.insert(0, 0)
B_minus.insert(0, 0)

Plotting

In [24]:
x = np.arange(-1, len(B_plus)-1, 1)

# UC Berkeley colors
berkeley_blue = "#002676"
california_gold = "#FDB515"

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=x,
    y=B_plus,
    mode="lines",
    name="B⁺ (cost increase)",
    line=dict(color=berkeley_blue, width=3.5)
))

fig.add_trace(go.Scatter(
    x=x,
    y=B_minus,
    mode="lines",
    name="B⁻ (cost decrease)",
    line=dict(color=california_gold, width=3.5)
))

fig.update_layout(
    title="Cumulative Adjustment Functions",
    xaxis_title="Month After Cost Increase",
    yaxis_title="Pass-Through Rate",

    # ECONOMETRICA / QJE STYLE
    font=dict(
        family="Times New Roman",
        size=14,
        color="black"
    ),
    template="simple_white",

    # Axes styling
    xaxis=dict(
        showgrid=True,
        gridcolor="lightgray",
        zeroline=False,
        linecolor="black",
        linewidth=1,
        mirror=True,
        ticks="outside",
        tickwidth=1,
        tickcolor="black"
    ),
    yaxis=dict(
        showgrid=True,
        gridcolor="lightgray",
        zeroline=False,
        linecolor="black",
        linewidth=1,
        mirror=True,
        ticks="outside",
        tickwidth=1,
        tickcolor="black"
    ),

    # Legend OUTSIDE (right)
    legend=dict(
        x=1.02,
        y=1,
        xanchor="left",
        yanchor="top",
        font=dict(size=15),
        bgcolor="rgba(255,255,255,0)",
        bordercolor="rgba(0,0,0,0)"
    ),

    margin=dict(r=150),
    width=850,
    height=520
)

fig.show()

fig.write_image(FOLDER + "plots/cpi_ppi/Eggs_IV.png") #this line saves the plot
